In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
from datetime import datetime
pd.set_option('display.max_rows', 1000)
pd.options.display.max_colwidth = 1000

#to send emails
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#to make the conection with spreadsheets
import gspread
from google.oauth2.service_account import Credentials

#for passwords
import os
from dotenv import load_dotenv, find_dotenv

# A. p_acquisition

### SET THE CREDENTIALS

In [2]:
load_dotenv(find_dotenv("../credentials/.env"))
email_key = os.environ.get("EMAIL_KEY") #EMAIL PASSWORD

### Spreadsheet connection

In [3]:
def conection_spreadsheet(scopes, credentials_location):
    scopes = scopes
    credentials = Credentials.from_service_account_file(
        credentials_location,
        scopes=scopes
    )
    gc = gspread.authorize(credentials)
    #If we want to host our credentials in a server we have to do the next code:
    # gc = gspread.service_account(filename='https://www.path/to/the/downloaded/file.json')
    print("spreadsheet conection done")
    return gc

gc = conection_spreadsheet(["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"], '../credentials/credentials.json')

spreadsheet conection done


### SET THE URLS IN VARIABLES

In [4]:
#JOIN all the urls
def full_urls(gc):
    sheet_erg = gc.open("business_afi_scraping_df_single").worksheet("urls_ergonomia")
    urls_ergonomia = sheet_erg.col_values(1)  #Obtener todos los registros

    sheet_ofi = gc.open("business_afi_scraping_df_single").worksheet("urls_oficina")
    urls_oficina = sheet_ofi.col_values(1)  #Obtener todos los registros

    sheet_rod = gc.open("business_afi_scraping_df_single").worksheet("urls_rodilla")
    urls_rodilla = sheet_rod.col_values(1)  #Obtener todos los registros

    sheet_gm = gc.open("business_afi_scraping_df_single").worksheet("urls_gaming")
    urls_gaming = sheet_gm.col_values(1)  #Obtener todos los registros
    
    urls_products_list_list = urls_ergonomia + urls_oficina + urls_rodilla + urls_gaming
    return urls_products_list_list
    
urls_products_list = full_urls(gc)

### web scrapping

Obtain the HTML of all our URLs

In [5]:
def parsed_content(urls_products_list):
    parsed_products_content_list = [bs4.BeautifulSoup(requests.get(i).content, "html.parser") for i in urls_products_list]
    return parsed_products_content_list

parsed_products_content_list = parsed_content(urls_products_list)

Obtain the price info of its class

In [6]:
def parsed_price_class(parsed_products_content_list):
    parsed_products_price_class = [i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text for i in parsed_products_content_list]
    return parsed_products_price_class

parsed_products_price_class = parsed_price_class(parsed_products_content_list)

Obtain the final price

In [7]:
def product_price(parsed_products_content_list,urls_products_list):
    final_price_products_list =[]
    for i,e in zip(parsed_products_content_list, range(len(urls_products_list))):
        try:
            i.find_all("div",{"class":"wp-block-media-text__content"})[0]
            #if it fails the except is executed
        except:
            if  urls_products_list[e] == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/":
                price_supuesto_0 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_0_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_0)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_0_cleaned))
            else:
                final_price_products_list.append(-1)
                print("revisar funcion price, valor con -1")
        else:
            if i.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip() == "No products found.":
                final_price_products_list.append(0)
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text == "":
                final_price_products_list.append(0)
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text != "":
                price_supuesto_1 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_1_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_1)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_1_cleaned))
            else:
                print("revisar funcion price")
    return final_price_products_list

final_price_products_list = product_price(parsed_products_content_list,urls_products_list)

Obtain if the product is out of stock or discontinued

In [8]:
def product_status(parsed_products_content_list,urls_products_list):
    final_price_products_status =[]
    for i,e in zip(parsed_products_content_list, range(len(urls_products_list))):
        try:
            i.find_all("div",{"class":"wp-block-media-text__content"})[0]
            #if it fails the except is executed
        except:
            if  urls_products_list[e] == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/":
                final_price_products_status.append("correcto")
            else:
                print("revisar funcion price, valor con -1")
                final_price_products_status.append("revisar")
        else:
            if i.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip() == "No products found.":
                final_price_products_status.append("descatalogado")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text == "":
                final_price_products_status.append("sin_stock")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text != "":
                final_price_products_status.append("correcto")
            else:
                print("revisar funcion price")
    return final_price_products_status

final_price_products_status = product_status(parsed_products_content_list,urls_products_list)

Create a function to handle possible erros in the product information

In [9]:
def handling_error_vars_product(i,text):
    try:
        str(i).split(text)[1]
    except:
        return "none"
    else:
        return str(i).split(text)[1].split("';")[0].strip()

Obtain the NAME of the different products

In [10]:
def product_name(parsed_products_content_list):
    
    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
    
    final_name_products = [handling_error_vars_product(i,"ficha_product_name='") for i in parsed_products_content_list]
    return final_name_products

final_name_products = product_name(parsed_products_content_list)

Obtain the ID of the different products

In [11]:
def product_id(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_id_products = [handling_error_vars_product(i,"ficha_product_id='") for i in parsed_products_content_list]
    return final_id_products

final_id_products = product_id(parsed_products_content_list)

Obtain the BRAND of the different products

In [12]:
def product_brand(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
    
    final_brand_products = [handling_error_vars_product(i,"ficha_product_brand='") for i in parsed_products_content_list]
    return final_brand_products

final_brand_products = product_brand(parsed_products_content_list)

Obtain the DATE of the current day in different formats

1. With HYPHEN. Ex: 2022-04-14

In [13]:
def product_date_hyphen(urls_products_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_hyphen_products = [datetime.today().strftime('%Y-%m-%d') for i in range(len(urls_products_list))]
    return final_date_hyphen_products

final_date_hyphen_products = product_date_hyphen(urls_products_list)

2. With SLASH. Ex: 2022/04/14

In [14]:
def product_date_slash(urls_products_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_slash_products = [datetime.today().strftime('%Y/%m/%d') for i in range(len(urls_products_list))]
    return final_date_slash_products

final_date_slash_products = product_date_slash(urls_products_list)

3. Without symbols. Ex: 20220414

In [15]:
def product_date_number(urls_products_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_number_products = [int(datetime.today().strftime('%Y%m%d')) for i in range(len(urls_products_list))]
    return final_date_number_products

final_date_number_products = product_date_number(urls_products_list)

# B. p_wrangling

## Create the DATAFRAME

### a) Full DataFrame with a daily injection of the new scraped data

#### Step 1: Create or define the dataFrame

 - From a CSV if it exists
 - If not, from spreadsheets
 - If not, from this notebook
 - If not, we have to create a empty dataFrame

In [16]:
def main_df(location,gc):

#2. If not, from spreadsheets    
        try:
            sheet = gc.open("business_afi_scraping_df_single").sheet1  #Abrir spreadhseet
            data_from_spreadsheets = sheet.get_all_records()  #Obtener todos los registros
            df_single = pd.DataFrame(data_from_spreadsheets)
            print("df_single created through spreadsheet")
            return df_single
        
#3. If not, from this notebook
        except: #if the except is executed, the try has given an error, so let's see if it was already created in the notebook       
            try:
                df_single.head()
                print("df_single created through this notebook")
                return df_single
            
#4. If not, we have to create a empty dataFrame
            except: #if it gives an error, it does not exist in the notebook and we will create it from scratch
                columns = ["date_hyphen","date_slash","date_number","product_name","product_id","product_brand","price","status","url"]
                df_single = pd.DataFrame(columns=columns)
                print("df_single created through zero")
                return df_single
            else: #if there is no error, it exists in the notebook and we will do nothing
                pass
            
        else: #if it doesn't give an error, it exists in spreadsheets and we won't do anything
            pass

df_single = main_df("../files/df_single.csv",gc)

df_single created through spreadsheet


#### Step 2: Append new records in the df_single dataFrame - pd.concat()

In [17]:
def df_append_new_files(final_date_hyphen_products,final_date_slash_products,final_date_number_products,final_name_products,final_id_products,final_brand_products,final_price_products_list,final_price_products_status,urls_products_list):
    df_append_new_files = pd.DataFrame({
        "date_hyphen":final_date_hyphen_products,
        "date_slash":final_date_slash_products,
        "date_number":final_date_number_products,
        "product_name":final_name_products,
        "product_id":final_id_products,
        "product_brand":final_brand_products,
        "price":final_price_products_list,
        "status":final_price_products_status,
        "url": urls_products_list
    })
    df_append_new_files["price"] = df_append_new_files["price"].astype("int64")
    
    return df_append_new_files

df_append_new_files = df_append_new_files(final_date_hyphen_products,final_date_slash_products,final_date_number_products,final_name_products,final_id_products,final_brand_products,final_price_products_list,final_price_products_status,urls_products_list)

In [18]:
def concat_df(df_single,df_append_new_files):
    if df_single.empty == True:
        df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)
        return df_single
    elif df_append_new_files["date_hyphen"][0] == df_single["date_hyphen"][len(df_single)-1]:
        return df_single
    elif df_append_new_files["date_hyphen"][0] != df_single["date_hyphen"][len(df_single)-1]:
        df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)
        return df_single
    
df_single = concat_df(df_single,df_append_new_files)

### b) Clean and Prepare the dataFrame

In [19]:
def corregir_gam_name(url,product_name):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_name == "none":
        return "intimate wm heat  Racing Silla Gamer"
    else:
        return product_name
        
def corregir_gam_id(url,product_id):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_id == "none":
        return "B075CK3GVJ"
    else:
        return product_id
        
def corregir_gam_brand(url,product_brand):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_brand == "none":
        return "intimate wm heat"
    else:
        return product_brand

    
def corregir_gam_name_hbada_rep(url,product_name):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_name == "none":
        return "Hbada B07V51M94R"
    else:
        return product_name
        
def corregir_gam_id_hbada_rep(url,product_id):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_id == "none":
        return "B07V51M94R"
    else:
        return product_id
        
def corregir_gam_brand_hbada_rep(url,product_brand):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_brand == "none":
        return "Hbada"
    else:
        return product_brand

In [20]:
def correction_df_single(df_single):
    #intimate wm heat  Racing Silla Gamer
    df_single["product_name"] = df_single.apply(lambda df_single: corregir_gam_name(df_single["url"], df_single["product_name"]), axis=1)
    df_single["product_id"] = df_single.apply(lambda df_single: corregir_gam_id(df_single["url"], df_single["product_id"]), axis=1)
    df_single["product_brand"] = df_single.apply(lambda df_single: corregir_gam_brand(df_single["url"], df_single["product_brand"]), axis=1)

    #Hbada B07V51M94R
    df_single["product_name"] = df_single.apply(lambda df_single: corregir_gam_name_hbada_rep(df_single["url"], df_single["product_name"]), axis=1)
    df_single["product_id"] = df_single.apply(lambda df_single: corregir_gam_id_hbada_rep(df_single["url"], df_single["product_id"]), axis=1)
    df_single["product_brand"] = df_single.apply(lambda df_single: corregir_gam_brand_hbada_rep(df_single["url"], df_single["product_brand"]), axis=1)

    return df_single

df_single = correction_df_single(df_single)

### c) Create the price main of each product (for the total period; for the last 7 days; for the last 30 days)

In [21]:
#product mean (TOTAL DAYS)
def product_mean_dic(df_single):
    product_id_list_mean = df_single["product_id"].unique().tolist()
    mean_dic = {}
    for i in product_id_list_mean:
        if str(df_single[df_single["product_id"] == i].loc[df_single["price"] > 0]["price"].mean()) == "nan":
            mean_dic[i] = 0
        else:
            mean_dic[i] = int(round(df_single[df_single["product_id"] == i].loc[df_single["price"] > 0]["price"].mean(),0))
    return mean_dic

product_mean = product_mean_dic(df_single### b) Clean and Prepare the dataFrame
)

#product mean (LAST 7 DAYS)
def product_mean_dic7(df_single):
    product_id_list_mean7 = df_single["product_id"].unique().tolist()
    mean_dic7 = {}
    for i in product_id_list_mean7:
        if str(df_single[df_single["product_id"] == i].iloc[-7:].loc[df_single["price"] > 0]["price"].mean()) == "nan":
            mean_dic7[i] = 0
        else:
            mean_dic7[i] = int(round(df_single[df_single["product_id"] == i].iloc[-7:].loc[df_single["price"] > 0]["price"].mean(),0))
    return mean_dic7

product_mean7 = product_mean_dic7(df_single### b) Clean and Prepare the dataFrame
)

#product mean (LAST 30 DAYS)
def product_mean_dic30(df_single):
    product_id_list_mean30 = df_single["product_id"].unique().tolist()
    mean_dic30 = {}
    for i in product_id_list_mean30:
        if str(df_single[df_single["product_id"] == i].iloc[-30:].loc[df_single["price"] > 0]["price"].mean()) == "nan":
            mean_dic30[i] = 0
        else:
            mean_dic30[i] = int(round(df_single[df_single["product_id"] == i].iloc[-30:].loc[df_single["price"] > 0]["price"].mean(),0))
    return mean_dic30

product_mean30 = product_mean_dic30(df_single### b) Clean and Prepare the dataFrame
)

In [22]:
def product_mean_dic_to_df(df_single,product_mean):
    df_single["product_mean"] = df_single["product_id"].apply(lambda x: product_mean[x])
    return df_single

df_single = product_mean_dic_to_df(df_single,product_mean)

def product_mean_dic_to_df7(df_single,product_mean7):
    df_single["product_mean7"] = df_single["product_id"].apply(lambda x: product_mean7[x])
    return df_single

df_single = product_mean_dic_to_df7(df_single,product_mean7)

def product_mean_dic_to_df30(df_single,product_mean30):
    df_single["product_mean30"] = df_single["product_id"].apply(lambda x: product_mean30[x])
    return df_single

df_single = product_mean_dic_to_df30(df_single,product_mean30)

In [23]:
def product_mean_status_func_values(product_price,product_mean):
    if product_price < product_mean and product_price > 0:
        return "precio por debajo de la media"
    elif product_price > product_mean and product_price > 0:
        return "precio por encima de la media"
    elif product_price == product_mean and product_price > 0:
        return "precio igual que la media"
    elif product_price == 0:
        return "producto descatalogado o sin stock"
    else:
        return "revisar, aviso"

In [24]:
def product_mean_status_func_apply(df_single):
    df_single["product_mean_status"] = df_single.apply(lambda x: product_mean_status_func_values(x["price"], x["product_mean"]), axis=1)
    return df_single

df_single = product_mean_status_func_apply(df_single)

In [25]:
def categories_url(product_url):
    if "/ergonomia/sillas-ergonomicas/review-individual/" in product_url:
        return "ergonomia"
    elif "/oficina-y-escritorio/sillas-de-oficina/review-individual/" in product_url:
        return "oficina"
    elif "/ergonomia/sillas-de-rodillas/review-individual/" in product_url:
        return "rodilla"
    elif "/gaming/sillas-gaming/review-individual/" in product_url:
        return "gaming"
    else:
        return "alerta sin categoria"

In [26]:
def product_category(df_single):
    df_single["product_category"] = df_single["url"].apply(categories_url)
    return df_single

df_single = product_category(df_single)

### e) Create a "count" column

In [27]:
def product_count(df_single):
    df_single["product_count"] = 1
    return df_single

df_single = product_count(df_single)

### f) Limit the the amount of data only to a year

In [28]:
def acotar_df_un_ano(df_single):
    if len(np.sort(df_single["date_number"].unique())) > 366:
        df_single = df_single[df_single["date_number"] > np.sort(df_single["date_number"].unique())[-366]]
        return df_single
    else:
        df_single = df_single
        return df_single
    
df_single = acotar_df_un_ano(df_single)

# D. p_reporting

We are going to send the different DataFrames to spreadsheet

#### INDEX LIBRARY
https://docs.gspread.org/en/latest/

#### USER GUIDE
https://docs.gspread.org/en/latest/user-guide.html

The conection has been done in the "p_wrangling" module. The output is located in the variable "gc"

In [29]:
def update_spreadsheet(gc, spreadsheet_name, worksheet_name, dataframe):
    #Open the spreadhseet
    sheet = gc.open(spreadsheet_name).worksheet(worksheet_name)
    
    #Clear and Update the Worksheet
    sheet.clear()
    sheet.update('A1:O1',[dataframe.columns.tolist()])
    sheet.update('A2:O' + str(len(dataframe)+1), dataframe.values.tolist())
    
    return "worksheet updated"

In [30]:
#sheet1 - df_single
update_spreadsheet(gc, "business_afi_scraping_df_single", "df_single", df_single)

'worksheet updated'

In [31]:
#sheet2 - df_append_new_files
update_spreadsheet(gc, "business_afi_scraping_last_day_files", "df_append_new_files", df_append_new_files)

'worksheet updated'

In [32]:
#sheet3 - out_of_stock_df
def out_of_stock_spreadsheet(df_append_new_files,gc):
    out_of_stock_df = df_append_new_files[df_append_new_files["status"] != "correcto"]
    update_spreadsheet(gc, "business_afi_scraping_last_day_files", "out_of_stock_df", out_of_stock_df)
    
    return "worksheet updated"
out_of_stock_spreadsheet(df_append_new_files,gc)

'worksheet updated'

In [33]:
#sheet4 - none_values_df
def non_values_spreadsheet(df_single):
    none_values = df_single[df_single["product_name"]=="none"].any().unique().tolist()
    if none_values == [True]:
        none_values_df = df_single[df_single["product_name"]=="none"]
        update_spreadsheet(gc, "business_afi_scraping_last_day_files", "none_values_df", none_values_df)
        
        return "worksheet updated"
    else:
        return "worksheet updated, but there is no any none value"
    
non_values_spreadsheet(df_single)

'worksheet updated, but there is no any none value'

In [34]:
#df_single.head()

In [35]:
#df_single.tail(5)

In [36]:
#len(df_single)